---

<small>

* `Few-shot 프롬프팅`
  * `Few-shot 프롬프팅`: 소수의 예시를 제공하여 모델이 특정 작업의 형식과 의도를 학습하게 하는 방법.
  * `Example Selector`: 여러 예시 중 입력과 **가장 유사한 예시** 를 **자동으로 선택** 하는 기능.

* `벡터스토어`와 `유사도 검색`
  * `Vector Store (Chroma)`: 텍스트를 `벡터(숫자 배열)`로 `변환`하여 저장하는 데이터베이스.
  * `Embedding Model`: 텍스트를 벡터로 변환하는 모델.
  * `유사도` 검색: 벡터스토어에서 입력된 텍스트와 `가장 유사한 벡터(예시)`를 찾는 과정.
  
* 모델의 `출력 제어`
  * `temperature`: 모델의 창의성 또는 무작위성을 조절하는 매개변수. 값이 높을수록 예측 불가능한 결과가 나옴.
  * `max_output_tokens`: 모델이 생성할 수 있는 최대 응답 길이.
  * `seed`: 모델의 무작위성을 고정하여 항상 동일한 결과가 나오도록 하는 매개변수.

---

<small>

* 기본 설정 (기존과 동일)

In [ ]:
# 기본 모듈 임포트
import os
import asyncio
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                                                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

In [ ]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* --- LangSmith 환경 변수 확인 ---
* ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
* ✅ LangSmith 프로젝트: 'LangChain-prantice'
* ✅ LangSmith API Key: 설정됨
*   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

In [ ]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트


print("\n--- LangChain 체인 설정 ---")                                # --- LangChain 체인 설정 ---

In [ ]:
# LLM 객체 정의하기

# model1 = gemini-1.5-flash
# model2 = gemini-2.5-falsh-lite

try:
    model1 = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-1.5-flash",
    )
    print("✅ gemini-1.5-flash 호출 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GEMINI_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ gemini-2.5-flash 호출 성공.


In [ ]:
# model2 = gemini-2.5-falsh-lite

try:
    model2 = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-2.5-flash-lite",
    )
    print("✅ gemini-2.5-flash-lite 호출 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GEMINI_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ gemini-2.5-flash-lite 호출 성공.

---

### (2) **`Example Seletor`**

#### ➀ `Example Selector`
* 예제가 많은 경우 프롬프트에 포함할 예제를 선택 작업을 담당하는 클래스
  * *[참고](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/)*


#### ➁ **`Example Selector Types`**
* 
* | 이름  | 설명                                           |
  |:-----|:----------------------------------------------|
  | **유사**  | 입력과 예시 사이의 의미적 유사성을 사용하여 어떤 예시를 선택할지 결정합니다.  |
  | `Similarity` | Uses semantic similarity between inputs and examples to decide which examples to choose.    |
  | **MMR** | 입력과 예시 간의 최대 한계 관련성을 사용하여 어떤 예시를 선택할지 결정합니다. |
  | `MMR`        | Uses Max Marginal Relevance between inputs and examples to decide which examples to choose. |
  | **길이**  | 특정 길이 내에 얼마나 많은 것이 들어갈 수 있는지에 따라 예를 선택합니다.   |
  | `Length`     | Selects examples based on how many can fit within a certain length                          |
  | **엔그램** | 입력과 예제 사이의 ngram 중복을 사용하여 어떤 예제를 선택할지 결정합니다. |
  | `Ngram`      | Uses ngram overlap between inputs and examples to decide which examples to choose.          |


<small>

* 교재 속 코드 풀이

    ```<python>
        from langchain_core.example_selectors import (    # 의미적 유사성을 기반으로 예시를 선택하는 클래스
            SemanticSimilarityExampleSelector,            # 사용자의 질문과 가장 '비슷한 의미'를 가진 예시를 찾아줌
        )
        from langchain_openai import OpenAIEmbeddings
        from langchain_chroma import Chroma

        # 예시들을 벡터(숫자 배열)로 변환하는 '임베딩 모델' 정의하기
        # (참고: 이 코드를 실행하려면 OpenAI API Key 필요)
        embedding_model = OpenAIEmbeddings()                    # 임베딩 모델 = 텍스트의 의미를 숫자로 표현하는 역할 수행

        # 벡터 데이터베이스(Vector DB) 생성하기
        chroma_vector_store = Chroma("example_selector", embedding_model)    # Chroma = 벡터들을 저장하고 빠르게 검색
                                    # "example_selector"라는 이름의 저장소 = 임베딩된 예시들을 보관
        

        # SemanticSimilarityExampleSelector 객체 생성 = 예시 선택의 모든 과정을 관리
        example_selector = SemanticSimilarityExampleSelector.from_examples(
            examples,                                           # [1] 선택 가능한 전체 예시 목록
            embedding_model,                                    # [2] 의미적 유사성을 측정하기 위해 사용할 임베딩 모델
            Chroma,                                             # [3] 임베딩을 저장하고 검색을 수행할 벡터 저장소 클래스
            k=1,                                                # [4] 사용자의 입력과 가장 유사한 예시를 몇 개 선택할지 지정 (여기서는 1개)
        )

        # 입력과 가장 유사한 예시 선택
        # 'question'이라는 키에 사용자의 실제 질문을 담아 전달
        selected_examples = example_selector.select_examples({"question": question})

        # 이 부분은 'question' 변수가 미리 정의되어 있다는 가정하에 작동
        question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
        print(f"입력에 가장 유사한 예시:\n{question}\n")

        # 선택된 예시를 순회하며 질문과 답변을 출력
        for example in selected_examples:
            print(f'question:\n{example["question"]}')
            print(f'answer:\n{example["answer"]}')
    ```

In [ ]:
# 필요한 라이브러리 불러오기
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 불러오기


# --- 1. Few-shot 예시 데이터 정의 ---
# 모델이 학습할 예시들(examples)을 파이썬 리스트 형태로 정의
# 각 예시들 = 'question'과 'answer' 키를 가진 딕셔너리 형태
# 이 예시들을 통해 모델은 질문에 답하는 방식 → 학습

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

# --- 2. 예시 포맷 정의 ---
# PromptTemplate 대신 ChatPromptTemplate 사용하기
# "human"과 "ai" 역할을 명확히 지정하여 예시의 형식을 정의하기
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "질문: {question}"),
        ("ai", "{answer}"),
    ]
)

# --- 3. Few-shot 예시 템플릿 생성 ---
# ChatPromptTemplate 기반의 예시를 다루는 FewShotChatMessagePromptTemplate를 사용
# 이 클래스가 어제 발생한 'AttributeError'를 해결
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# --- 4. 최종 프롬프트 구성 ---
# 전체 프롬프트 = `system` 메시지 + `few_shot_prompt` + `human` 메시지로 구성
# 모델에게 `역할`과 `예시``, 그리고 `새로운 질문`을 명확하게 전달 가능
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음은 몇 가지 질문과 답변 예시입니다. 이 형식을 따라 새로운 질문에 답해주세요.\n\n"),
    few_shot_prompt,
    ("human", "질문: {input}\n"),
])


# --- 5. LLM (대규모 언어 모델) 객체 정의 ---
# LangChain에서 Gemini 모델을 호출하는 객체 만들기
# 'temperature'를 0으로 설정하여 모델이 예시 패턴을 엄격하게 따르도록 유도
gemini_lc = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

# --- 6. LangChain Expression Language (LCEL)을 사용한 체인(Chain) 구성 ---
# 프롬프트 → LLM → 결과 파싱(parsing) 순서로 작업을 연결
# StrOutputParser는 모델의 응답을 단순한 문자열로 변환해 줍니다.
chain_lc = final_prompt | gemini_lc | StrOutputParser()

# --- 7. 새로운 질문으로 체인 실행 ---
# 'chain.invoke'를 호출하여 Few-shot 프롬프트를 모델에 전달하고 응답을 받기
new_question = "제임스 고슬링과 귀도 반 로섬 중 누가 파이썬을 만들었나요?"
response = chain_lc.invoke({"input": new_question})

# --- 8. 결과 출력 ---
print("--- Few-shot 프롬프트를 적용한 모델의 응답 ---")
print(response)

<small>

* 셀 출력 (1.3s)

    ```<plaintext>
        --- Few-shot 프롬프트를 적용한 모델의 응답 ---
        이 질문에 추가 질문이 필요한가요: 예.
        추가 질문: 제임스 고슬링은 어떤 프로그래밍 언어를 만들었나요?
        중간 답변: 제임스 고슬링은 자바를 만들었습니다.
        추가 질문: 귀도 반 로섬은 어떤 프로그래밍 언어를 만들었나요?
        중간 답변: 귀도 반 로섬은 파이썬을 만들었습니다.
        최종 답변은: 귀도 반 로섬
    ```

---

* `기본 SemanticSimilarityExampleSeletor`의 한계
  * 예시 **전체**(`instruction` + `input` + `answer`) → **벡터화** → **유사도** 계산
  * 결과적으로 사용자의 의도와 다른 예시가 선택되는 경우 발생
<br><br>

* **기본 인터페이스**

  ```<python>

    class BaseExampleSelector(ABC):
        """Interface for selecting examples to include in prompts."""

        @abstractmethod
        def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
            """Select which examples to use based on the inputs."""
            
        @abstractmethod
        def add_example(self, example: Dict[str, str]) -> Any:
            """Add new example to store."""

  ```

<small>

* 교안 코드 해석
<br><br>

* `examples` 목록
    * 이전 코드와 달리 `instruction`과 `input`으로 구성된 `여러 종류`의 `예시 데이터`
    * 회의록 작성 예시: `instructio`n에 **회의록 작성 전문가** 역할을 부여하고, `input`에 **회의 내용** 을, `answer`에 **완성된 회의록** 을 포함
    * 요약 전문가 예시: `instruction`에 **요약 전문가** 역할을 부여하고, `input`에 **긴 문서** 를, `answer`에 **요약된 내용** 을 포함
    * 문장 교정 예시: `instruction`에 **문장 교정 전문가** 역할을 부여하고, `input`에 **원문** 을, `answer`에 **교정된 문장** 을 포함

* `퓨샷` 예제와 `비슷한 셀렉터` 선택하기 - `Example Selector`가 **어떻게 작동**하는지 보여줌
  * `question 변수`
    * 회의록 작성을 요청하는 새로운 입력
    * `instruction`과 `input`을 `딕셔너리` 형태로 담고 있음

  * `example_selector.select_examples(question)`을 실행 → `Example Selector`는 `question`의 `내용(회의록 작성)`과 **가장 의미적으로 유사한 예시** 를 `examples 목록`에서 찾아냄

  * `출력 결과` 
    * 회의록 작성에 대한 `첫 번째 예시를 정확히 선택`한 것을 확인 가능
    * `Example Selector`가 단순히 키워드가 아닌 **작업의 의도를 파악** 하여 `적절한 예시`를 `찾아낸다`는 것을 의미

* `final_prompt`와 `chain`
  * `final_prompt` 
    * `system 역할`로 `전반적인 지시`를 내림
    * `few_shot_prompt`로 `Example Selector`가 `선택`한 `예시`를 `포함`
    * 마지막으로 `human` 역할로 `새로운 질문`을 `전달` = `human 프롬프트`는 **`instruction`과 `input`을 결합한 형태** 로 구성

  * `chain` = `final_prompt` | `llm`  → 연결 → 전체 파이프라인 완성

* 최종 출력 = `chain.stream(question)` 
  * 모델이 `instruction`과 `input`을 바탕으로 `Example Selector`가 선택한 예시의 형식을 따라 새로운 회의록을 성공적으로 생성했음을 보여줌

In [ ]:
# 필요한 라이브러리들을 불러오기
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma                                             # 터미널에서 먼저 pip install langchain-chroma
import os
from dotenv import load_dotenv
import uuid

# .env 파일에서 환경 변수 불러오기
load_dotenv()

# --- 1. Few-shot 예시 데이터 정의 (변경점 1: 예시의 구조와 내용이 변경됨) ---
# 다양한 작업(회의록, 요약, 교정)에 대한 예시를 담고 있음
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
    - 회의는 김수진 팀장의 개회사로 시작됨.
    - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.
...
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
...
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

# --- 2. Example Selector 설정 ---
# 텍스트를 벡터로 변환하는 임베딩 모델
# Gemini API 기반 모델 사용하기
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# (수정됨)
# Chroma.from_examples 메소드는 존재하지 않음
"""
# 예시를 저장하고 검색할 벡터 저장소 생성하기
vectorstore = Chroma.from_examples(
    examples,
    embeddings,
    collection_name="few-shot-examples-collection",
)
"""

# 예시 딕셔너리에서 검색에 사용할 텍스트와 메타데이터를 분리하여 Chroma.from_texts 메소드를 사용
texts_to_embed = [
    f'instruction: {ex["instruction"]}\ninput: {ex["input"]}' for ex in examples
]
metadatas = [
    { "instruction": ex["instruction"], "input": ex["input"], "answer": ex["answer"] } for ex in examples
]

# (수정됨) 매번 새로운 컬렉션을 생성하도록 collection_name을 동적으로 변경
# 벡터스토어 캐싱 문제 발생 해결 위함
collection_name = f"few-shot-examples-collection-{uuid.uuid4()}"



# 예시 텍스트와 메타데이터를 사용하여 벡터 저장소 생성
vectorstore = Chroma.from_texts(
    texts=texts_to_embed,
    embedding=embeddings,
    metadatas=metadatas,
    #collection_name="few-shot-examples-collection",
    collection_name=collection_name,
)


# SemanticSimilarityExampleSelector 생성하기
# 새로운 입력(질문)과 의미적으로 가장 유사한 예시를 찾아 반환함
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1                                                                 # 가장 유사한 예시 1개 선택
)

# --- 3. 예시 포맷 및 최종 프롬프트 구성 ---
# 예시 하나하나의 형식을 ChatPromptTemplate으로 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "지시: {instruction}\n입력: {input}"),
        ("ai", "{answer}"),
    ]
)

# FewShotChatMessagePromptTemplate을 사용해 예시들을 템플릿에 통합
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,                                      # (변경점 2: 'examples' 대신 'example_selector'를 사용)
)

# 최종 프롬프트
# system 메시지로 역할을 지정하고, few-shot 예시, 새로운 질문(human)을 포함
# (변경점 3: human 메시지의 구조가 instruction과 input을 포함하도록 변경됨)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "주어진 지시와 입력에 따라 작업을 수행하고 예시와 같은 형식으로 출력해주세요."),
        few_shot_prompt,
        ("human", "지시: {instruction}\n입력: {input}"),
    ]
)


# --- 4. LLM 객체 정의 및 체인 구성 ---
gemini_lc = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", 
    temperature=0.7,                      # temperature, max_output_tokens = 범용적 인자 = 그대로 사용 가능
    max_output_tokens=4096,               # gemini-2.5-flash-lite의 긴 글 생성시 권장되는 최소 max_tokens
    #model_kwargs={"seed": 42}             # 특정 모델에만 있거나 실험적인 기능안 model_kwargs라는 딕셔너리를 통해 전달해야 함
)
chain_lc2 = final_prompt | gemini_lc | StrOutputParser()


# --- 5. 새로운 질문으로 체인 실행 ---
# (변경점 4: 'new_question'이 instruction과 input을 가진 딕셔너리로 변경됨)
new_question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 'chain.invoke'를 호출하여 Few-shot 프롬프트를 모델에 전달하고 응답을 받기
response2 = chain_lc2.invoke(new_question)

# --- 6. 결과 출력 ---
print("--- Few-shot Example Selector를 적용한 모델의 응답 ---")
print(response2)

<small>

* 셀 출력 (2.9s)(max_tokens = 따로 지정하지 않음 = 기본 값 = 1,042)

     ```<plaintext>
        --- Few-shot Example Selector를 적용한 모델의 응답 ---
        회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
        일시: 2023년 12월 26일
        장소: ABC 기술 회사 회의실
        참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

        1. 개회
            - 회의는 최현수 프로젝트 매니저의 개회로 시작됨.
            - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 주간 진행 상황 검토 및 다가오는 마일스톤 계획 수립.
        ...

    ```

* 셀 출력 (4.2s)(max_tokesn = 4,096 )

    ```<markdown>
        --- Few-shot Example Selector를 적용한 모델의 응답 ---
        ## 회의록

        **일시:** 2023년 12월 26일
        **장소:** ABC 기술 회사 회의실
        **참석자:** 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

        ### 1. 회의 목적

        *   새로운 모바일 애플리케이션 프로젝트 주간 진행 상황 검토
        *   다가오는 마일스톤 계획 수립

        ### 2. 논의 내용

        *   **최현수 (프로젝트 매니저):**
            *   프로젝트 현재 진행 상황 전반에 대한 개요 발표
            *   전반적인 일정 준수 및 잠재적 위험 요소 공유
        *   **황지연 (주요 개발자):**
            *   주요 기능 개발 진행 상황 보고 (구체적인 기능 명시 필요)
            *   현재까지 완료된 개발 작업 및 발견된 기술적 이슈 공유
            *   다음 주 개발 목표 제시
        *   **김태영 (UI/UX 디자이너):**
            *   현재까지 진행된 UI/UX 디자인 작업 결과물 공유 (구체적인 화면 또는 기능 명시 필요)
            *   사용자 테스트 결과 및 피드백 반영 현황 보고
            *   다음 주 디자인 작업 계획 제시

        ### 3. 다음 주 목표

        *   (황지연) [구체적인 개발 목표 1], [구체적인 개발 목표 2] 완료
        *   (김태영) [구체적인 디자인 작업 1], [구체적인 디자인 작업 2] 완료
        *   (최현수) [구체적인 프로젝트 관리 업무 1] 진행

        ### 4. 기타

        *   (추가 논의 사항이 있다면 기록)

        ### 5. 폐회

        *   회의는 다음 주 회의 일정 확인 후 폐회함.
    ```

---

<small>

* 교안 코드 해석 = **핵심 = `SemanticSimilarityExampleSelector`의 한계와 극복 방법**

* 기존 `SemanticSimilarityExampleSelector`의 문제점
  * 유사도를 계산할 때 `instruction` (지시)과 `input` (입력)을 **모두 고려**
  * 따라서 `회의록을 작성해 주세요`라는 질문을 던졌을 때 → 내용이 더 유사한 **문장 교정** 예시를 가져오는 문제가 발생할 수 있음

    * 예시: `instruction`만 보면 **회의록**과 **문장 교정**은 전혀 다르지만, **`input`의 내용** 까지 포함하면 유사도가 높다고 오판할 가능성 있음

* **`CustomExampleSelector`를 통한 해결**: 
  * **`instruction`만 사용** → `유사도`를 `계산`하는 **커스텀 클래스** 사용
  * **질문의 의도에만 집중** → **가장 적합한 예시 정확하게 선택 가능**

    * 예시: `CustomExampleSelector` = **회의록을 작성해 주세요**라는 질문 → **당신은 회의록 작성 전문가입니다**라는 `instruction`을 가진 예시 선택
  * 이러한 커스텀 클래스를 사용함으로써, 모델이 올바른 예시를 참고하여 훨씬 더 정확하고 의도에 맞는 결과물을 생성 가능

In [ ]:
# 필요한 라이브러리 불러오기
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors.base import BaseExampleSelector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from typing import Dict, List, Any
import os
from dotenv import load_dotenv
import uuid

# 환경 변수 불러오기
load_dotenv()

# --- 1. instruction 만을 사용하는 커스텀 예제 선택기 클래스 정의 ---
class CustomInstructionExampleSelector(BaseExampleSelector):
    """지시(instruction)만 사용하여 유사도를 계산하는 커스텀 예제 선택기입니다."""

    def __init__(self, examples: List[Dict], embeddings: Any, k: int = 1):
        """
        초기화 함수.
        Args:
            examples (List[Dict]): few-shot 예시 데이터.
            embeddings (Any): 임베딩 모델 객체.
            k (int): 반환할 유사 예시의 개수.
        """
        self.k = k
        self.embeddings = embeddings

        # 텍스트와 메타데이터를 분리하여 Chroma 벡터스토어에 저장하기
        # 이 때, 텍스트는 instruction 만을 사용함
        texts_to_embed = [ex["instruction"] for ex in examples]
        metadatas = examples

        # 매번 새로운 컬렉션을 생성하여 캐싱 문제가 발생하지 않도록 하기
        collection_name = f"custom-few-shot-collection-{uuid.uuid4()}"
        
        self.vectorstore = Chroma.from_texts(
            texts=texts_to_embed,
            embedding=self.embeddings,
            metadatas=metadatas,
            collection_name=collection_name,
        )
        
        
    # [수정된 부분] 추상 클래스 규칙을 만족시키기 위해 'add_example' 메서드 추가
    # 이 메서드는 현재 코드에서 사용되지는 않지만, 정의해야만 객체 생성이 가능
    def add_example(self, example: Dict[str, str]) -> Any:
        """
        새로운 예시를 추가하는 메서드. 이 커스텀 셀렉터는 예시를 추가하는 기능을 지원하지 않습니다.
        """
        raise NotImplementedError(
            "This custom example selector does not support adding examples."
        )

    def select_examples(self, input_variables: Dict[str, str]) -> List[Dict]:
        """
        주어진 입력과 가장 유사한 예시를 검색합니다.
        Args:
            input_variables (Dict): 검색할 지시와 입력 정보.
        Returns:
            List[Dict]: 검색된 유사 예시 리스트.
        """
        instruction_to_search = input_variables["instruction"]
        # instruction 만을 사용해 유사도 검색을 수행하기
        results = self.vectorstore.similarity_search_with_score(
            instruction_to_search, k=self.k
        )
        # 검색된 결과에서 메타데이터(원본 예시)만 추출하기
        examples = [result[0].metadata for result in results]
        return examples

# --- 2. Few-shot 예시 데이터 정의 ---
# 다양한 작업(회의록, 요약, 교정)에 대한 예시
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
    - 회의는 김수진 팀장의 개회사로 시작됨.
    - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.
...
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
...
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

# --- 3. 커스텀 예제 선택기 설정 ---
# gemini 임베딩 모델 = 텍스트를 벡터로 변환하는 임베딩 모델
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 커스텀 예제 선택기를 생성하기
custom_selector = CustomInstructionExampleSelector(
    examples=examples,
    embeddings=embeddings,
    k=1                                                                 # 가장 유사한 예시 1개 선택
)

# --- 4. 예시 포맷 및 최종 프롬프트 구성 ---
# 예시 하나하나의 형식을 ChatPromptTemplate으로 정의하기
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "지시: {instruction}\n입력: {input}"),
        ("ai", "{answer}"),
    ]
)

# FewShotChatMessagePromptTemplate을 사용해 예시들을 템플릿에 통합하기
custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=custom_selector,                                   # 커스텀 예제 선택기 사용
)

# 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "주어진 지시와 입력에 따라 작업을 수행하고 예시와 같은 형식으로 출력해주세요."),
        custom_fewshot_prompt,
        ("human", "지시: {instruction}\n입력: {input}"),
    ]
)

# --- 5. LLM 객체 정의 및 체인 구성 ---
gemini_lc = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,                                                      # 무작위성 없이 일관된 결과 도출
    max_output_tokens=4092,
    model_kwargs={}                                                     # seed가 없으므로 빈 딕셔너리로 설정
)
chain_lc3 = final_prompt | gemini_lc | StrOutputParser()

# --- 6. 새로운 질문으로 체인 실행 및 결과 출력 ---
# 회의록 작성 예시
question_1 = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}
print("--- 회의록 작성 예시 결과 ---")
response_3 = chain_lc3.invoke(question_1)
print(response_3)
print("\n" + "="*50 + "\n")

# 문서 요약 예시
question_2 = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}
print("--- 문서 요약 예시 결과 ---")
response_3 = chain_lc3.invoke(question_2)
print(response_3)
print("\n" + "="*50 + "\n")

# 문장 교정 예시
question_3 = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}
print("--- 문장 교정 예시 결과 ---")
response_3 = chain_lc3.invoke(question_3)
print(response_3)

<small>

* ERROR 메시지

    ```<code>
      ---------------------------------------------------------------------------
      TypeError                                 Traceback (most recent call last)
      Cell In[20], line 105
          102 embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
          104 # 커스텀 예제 선택기를 생성하기
      --> 105 custom_selector = CustomInstructionExampleSelector(
          106     examples=examples,
          107     embeddings=embeddings,
          108     k=1                                                                 # 가장 유사한 예시 1개 선택
          109 )
          111 # --- 4. 예시 포맷 및 최종 프롬프트 구성 ---
          112 # 예시 하나하나의 형식을 ChatPromptTemplate으로 정의하기
          113 example_prompt = ChatPromptTemplate.from_messages(
          114     [
          115         ("human", "지시: {instruction}\n입력: {input}"),
          116         ("ai", "{answer}"),
          117     ]
          118 )

      TypeError: Can't instantiate abstract class CustomInstructionExampleSelector without an implementation for abstract method 'add_example'
    ```

* ERROR 발생 이유
  * `CustomInstructionExampleSelector`라는 클래스가 `BaseExampleSelecto`r라는 **추상 클래스(abstract class)**를 상속받았는데, `BaseExampleSelector`가 반드시 구현해야 한다고 정해놓은 `필수 메서드`인 `add_example`을 빠뜨렸기 때문에 발생
  * 즉, `BaseExampleSelector`는 **나를 상속받으려면 `add_example`이라는 기능을 무조건 만들어야 해!**라고 규칙을 정해둠

* 해결 방법
  * `CustomInstructionExampleSelector` 클래스에 **`add_example` 메서드를 추가** 해야 함
  * 이 코드에서는 이 메서드를 직접 사용하지는 않지만, 추상 클래스의 규칙을 만족시키기 위해 반드시 정의해두어야 정상 작동함

---

<small>

* 셀 출력 (7.2s)(max_tokens=4092)
  
    ```<markdown>
        --- 회의록 작성 예시 결과 ---
        ## ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의록

        **일시:** 2023년 12월 26일
        **장소:** 회의실 3
        **참석자:**
        *   최현수 (프로젝트 매니저)
        *   황지연 (주요 개발자)
        *   김태영 (UI/UX 디자이너)

        **회의 목적:**
        *   신규 모바일 애플리케이션 프로젝트 진행 상황 검토
        *   다가오는 마일스톤 계획 수립

        **주요 논의 내용:**

        1.  **각 팀원별 진행 상황 보고:**
            *   **최현수 (프로젝트 매니저):** 전체 프로젝트 일정 및 예산 현황 공유. 현재까지 계획대로 진행 중이며, 특이사항 없음.
            *   **황지연 (주요 개발자):** 현재까지 개발된 기능 목록 및 테스트 결과 보고. 일부 기능에 대한 추가 테스트 필요.
            *   **김태영 (UI/UX 디자이너):** 현재까지 디자인된 화면 및 사용자 흐름 검토. 사용자 피드백 반영하여 일부 수정 예정.

        2.  **다가오는 마일스톤 계획 수립:**
            *   다음 주까지 완료해야 할 주요 작업 항목 확정.
            *   황지연 개발자는 사용자 인증 기능 개발 및 테스트 완료.
            *   김태영 디자이너는 메인 화면 및 사용자 프로필 화면 디자인 최종 확정 및 개발팀 전달.
            *   최현수 PM은 다음 주 회의에서 각 팀원의 진행 상황 점검 및 필요한 지원 제공.

        **결정 사항:**

        *   다음 주까지 황지연 개발자는 사용자 인증 기능 개발 및 테스트 완료.
        *   다음 주까지 김태영 디자이너는 메인 화면 및 사용자 프로필 화면 디자인 최종 확정 및 개발팀 전달.
        *   다음 주 회의에서 각 팀원의 진행 상황 점검 및 필요한 지원 제공.

        **차기 회의:**
        *   일시: 2024년 1월 2일 (화)
        *   장소: 회의실 3
        *   주요 안건: 지난 주 목표 달성 여부 확인 및 다음 주 작업 계획 수립

        **작성자:** 최현수 (프로젝트 매니저)

        ==================================================

        --- 문서 요약 예시 결과 ---
        이 문서는 2023년 글로벌 경제 전망에 대한 30페이지 분량의 보고서로, 현재 세계 경제 상황, 주요 국가 성장률, 무역 동향, 향후 경제 예측 및 다양한 요인이 경제에 미칠 영향을 분석합니다.

        ==================================================

        --- 문장 교정 예시 결과 ---
        회사는 올해 매출이 증가할 것으로 예상하며, 이는 새로운 전략이 성공적으로 작동하고 있음을 시사한다.
    ```